# Modélisation des autorisations de crédits

## Comparaison des perfs de différents modèles

L'objectif est d'avoir un premier aperçu de la performance de différents types de modèles avec des paramètres standards.

### Chargement des librairies

In [1]:
# Librairies Data Science
import pandas as pd
import numpy as np
from collections import Counter
import time

In [2]:
# Scikit-Learn
from sklearn.model_selection import (cross_validate,
                                     StratifiedKFold)
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier

In [3]:
# Imbalanced-Learn
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler

### Chargement des données

In [4]:
# Chargement des données intégrales déjà retraitées et séparées en train & test sets
# 11 features sélectionnées + undersampling + imputation des valeurs manquantes
X_train = pd.read_csv('../data_models/X_train_imputed.csv', index_col=0)
X_test = pd.read_csv('../data_models/X_test_imputed.csv', index_col=0)

In [5]:
# Isolation de la target
y_train = X_train['TARGET']
y_test = X_test['TARGET']
# Supprimer la target des features
X_train = X_train.drop(columns='TARGET')
X_test = X_test.drop(columns='TARGET')

In [7]:
# Comptage des classes
Counter(y_train)

Counter({0.0: 16792, 1.0: 16700})

In [8]:
Counter(y_test)

Counter({0.0: 7131, 1.0: 7223})

On voit que les classes ont bien été égalisées.

# Tests de différents modèles

Sélection de modèles adaptés aux données de grande taille. Pour plus de détails, voir note méthodologique. Les SVM et les KNN ne sont pas étudiés car ils ne sont pas adaptés aux datasets de grande taille.

Modèles sélectionnés: Naive Bayes, régression logistique, arbre de décision, XGboost et Random Forest.

In [9]:
# Entrainement sur le train set initial complet
X = pd.concat([X_train,X_test])
y = pd.concat([y_train,y_test])

In [10]:
models = [GaussianNB(),
          RidgeClassifier(),
          DecisionTreeClassifier(),
          XGBClassifier(),
          RandomForestClassifier()]

names = ["GaussianNB()", "Ridge", "Decision Tree",
         'XGBoost', 'RandomForest']

# Calcul de la performance des modèles via une boucle unique
for model, name in zip(models, names):
    print (name +' with balanced class weights')
    print("-----------------")
    start = time.time()
    model_ = make_pipeline(StandardScaler(),
                           model)
    scoring = {'recall': 'recall',
               'auc': 'roc_auc',
               'f1': 'f1',
               }
    cv = StratifiedKFold(n_splits=3) 
    scores = cross_validate(model_,
                            X, y,
                            scoring=scoring,
                            cv=cv,
                            return_train_score=True)
    for key, value in scores.items():
        print(key, ' :', round(value.mean(), 2) )
    print("-----------------")

GaussianNB() with balanced class weights
-----------------
fit_time  : 0.02
score_time  : 0.03
test_recall  : 0.72
train_recall  : 0.72
test_auc  : 0.79
train_auc  : 0.79
test_f1  : 0.72
train_f1  : 0.72
-----------------
Ridge with balanced class weights
-----------------
fit_time  : 0.03
score_time  : 0.04
test_recall  : 0.69
train_recall  : 0.69
test_auc  : 0.8
train_auc  : 0.8
test_f1  : 0.72
train_f1  : 0.72
-----------------
Decision Tree with balanced class weights
-----------------
fit_time  : 0.29
score_time  : 0.02
test_recall  : 0.72
train_recall  : 1.0
test_auc  : 0.69
train_auc  : 1.0
test_f1  : 0.7
train_f1  : 1.0
-----------------
XGBoost with balanced class weights
-----------------
fit_time  : 1.11
score_time  : 0.04
test_recall  : 0.78
train_recall  : 0.87
test_auc  : 0.86
train_auc  : 0.95
test_f1  : 0.76
train_f1  : 0.87
-----------------
RandomForest with balanced class weights
-----------------
fit_time  : 5.06
score_time  : 0.48
test_recall  : 0.76
train_recall  

Les modèles simples de type Naive Bayes et régression logistique donnent des performances très correctes (recall, auc_roc) avec des temps de calcul très courts. Les performances similaires à celles des méthodes plus complexes.   
Les temps de calcul sont plus longs pour les arbres (RandomForest et XGBoost) mais ces méthodes permettent d'améliorer les perfomances. Ils ont néanmoins tendance à overfitter (performance sur le test set inférieures à celles sur le train set). XGBoost semble être un bon compris entre performance et vitesse.

Etudier le notebook https://colab.research.google.com/drive/1pjPzsw_uZew-Zcz646JTkRDhF2GkPk0N#scrollTo=rrTZ874kv7Hm pour aller plus loin